In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 1. Hyper-parameters and Dataset

**Image preprocessing**: converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = '/content/drive/My Drive/PyTorch/Github_Series/01-basics/'

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=data_dir,
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root=data_dir,
                                          train=False,
                                          transform=transforms.ToTensor(),
                                          download=True)

# MNIST dataset loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

# 2. Modeling and Training

In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
  
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [ ]:
# Initialize the model
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
  for batch_index, (images, labels) in enumerate(train_loader):
    # Reshpae into (batch_size, input_size)
    input = images.reshape(-1, input_size).to(device)   # It is necessary to have both the model and the data on the same device
    labels = labels.to(device)
    # Feedforward
    output = model.forward(input)
    # Loss
    loss = loss_fn(output, labels)
    # Backward propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (batch_index+1) % 100 == 0:
      print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'
            .format(epoch+1, num_epochs, batch_index+1, total_step, loss.item()))

Epoch: [1/5], Step: [100/600], Loss: 0.2662
Epoch: [1/5], Step: [200/600], Loss: 0.2029
Epoch: [1/5], Step: [300/600], Loss: 0.3094
Epoch: [1/5], Step: [400/600], Loss: 0.2401
Epoch: [1/5], Step: [500/600], Loss: 0.0903
Epoch: [1/5], Step: [600/600], Loss: 0.1478
Epoch: [2/5], Step: [100/600], Loss: 0.1168
Epoch: [2/5], Step: [200/600], Loss: 0.1536
Epoch: [2/5], Step: [300/600], Loss: 0.0421
Epoch: [2/5], Step: [400/600], Loss: 0.0558
Epoch: [2/5], Step: [500/600], Loss: 0.1232
Epoch: [2/5], Step: [600/600], Loss: 0.0645
Epoch: [3/5], Step: [100/600], Loss: 0.0399
Epoch: [3/5], Step: [200/600], Loss: 0.0567
Epoch: [3/5], Step: [300/600], Loss: 0.0426
Epoch: [3/5], Step: [400/600], Loss: 0.0397
Epoch: [3/5], Step: [500/600], Loss: 0.0523
Epoch: [3/5], Step: [600/600], Loss: 0.0385
Epoch: [4/5], Step: [100/600], Loss: 0.0868
Epoch: [4/5], Step: [200/600], Loss: 0.0454
Epoch: [4/5], Step: [300/600], Loss: 0.0556
Epoch: [4/5], Step: [400/600], Loss: 0.0813
Epoch: [4/5], Step: [500/600], L

# 3. Test the model

In [ ]:
# Test the model
with torch.no_grad():
  correct = 0
  total_cnt = 0
  for images, labels in test_loader:
    input = images.reshape(-1, input_size).to(device)
    labels = labels.to(device)
    output = model.forward(input)
    _, pred = torch.max(output, dim=1)    # (max_value, max_index)
    total_cnt += labels.size(0)
    correct += (pred == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total_cnt))

Accuracy of the network on the 10000 test images: 97.71 %
